In [16]:
import numpy as np
from PIL import Image
import cv2
from scipy import ndimage
from skimage import feature, filters

In [17]:
def load_and_preprocess(image_path, size=(30, 30)):
    """Load and preprocess image to grayscale."""
    img = Image.open(image_path)
    img = img.convert('L')  # Convert to grayscale
    img = img.resize(size, Image.Resampling.LANCZOS)
    return np.array(img)

def simple_threshold(img, threshold=127):
    """Simple threshold-based binarization."""
    return (img > threshold).astype(int)

def adaptive_threshold(img):
    """Adaptive thresholding using local regions."""
    img_uint8 = img.astype(np.uint8)
    binary = cv2.adaptiveThreshold(
        img_uint8, 1, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2
    )
    return binary

def otsu_threshold(img):
    """Otsu's method for optimal thresholding."""
    img_uint8 = img.astype(np.uint8)
    _, binary = cv2.threshold(img_uint8, 0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return binary

def edge_based(img):
    """Edge-based binarization using Canny edge detection."""
    edges = feature.canny(img/255.0, sigma=1)
    # Dilate edges slightly to make them more visible in 30x30
    return ndimage.binary_dilation(edges).astype(int)

def gradient_based(img):
    """Gradient-based binarization using Sobel filters."""
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)
    gradient_magnitude = np.sqrt(sobelx**2 + sobely**2)
    return (gradient_magnitude > np.mean(gradient_magnitude)).astype(int)

def local_variance(img):
    """Binarization based on local variance."""
    local_var = ndimage.generic_filter(img, np.var, size=3)
    return (local_var > np.mean(local_var)).astype(int)

def dither_floyd_steinberg(img):
    """Floyd-Steinberg dithering for better detail preservation."""
    img_copy = img.astype(float)
    height, width = img_copy.shape
    
    for y in range(height-1):
        for x in range(width-1):
            old_pixel = img_copy[y, x]
            new_pixel = round(old_pixel/255.0) * 255
            img_copy[y, x] = new_pixel
            error = old_pixel - new_pixel
            
            img_copy[y, x+1] += error * 7/16
            img_copy[y+1, x-1] += error * 3/16
            img_copy[y+1, x] += error * 5/16
            img_copy[y+1, x+1] += error * 1/16
            
    return (img_copy > 127).astype(int)

def create_all_versions(image_path):
    """Create all binary versions of the image using different techniques."""
    img = load_and_preprocess(image_path)
    
    versions = {
        'simple': simple_threshold(img),
        'adaptive': adaptive_threshold(img),
        'otsu': otsu_threshold(img),
        'edge': edge_based(img),
        'gradient': gradient_based(img),
        'variance': local_variance(img),
        'dither': dither_floyd_steinberg(img)
    }
    
    return versions

def visualize_binary(binary_array):
    """Helper function to visualize binary array as ASCII art."""
    for row in binary_array:
        print(''.join(['█' if pixel else ' ' for pixel in row]))

def save_binary(binary_array, filename):
    """Save binary array as image."""
    Image.fromarray(binary_array * 255).convert('1').save(filename)

In [ ]:
image_path = "input_image.jpg"
    versions = create_all_versions(image_path)
    
    # Print ASCII preview of each version
    for name, binary in versions.items():
        print(f"\n{name.upper()} VERSION:")
        visualize_binary(binary)
        # Optionally save the binary image
        save_binary(binary, f"binary_{name}.png")